# Create Combined All Data - Fixed Quarter Dates

1. Credit Rating  
    * File: credit_ratings_on_fixed_quarter_dates.csv  
    * Columns: rating, symbol, rating agency, date, previous/next rating/date, type, change in rating ...  
    * Size: 9117 × 16  
    * 638 unique companies  
2. Earnings Call  
    * File: calls_short.csv  
    * Columns: company, sector, year, quarter, date, transcript  
    * Size: 24580 x 6  
    * 1322 unique companies  
3. Financial Data  
    * File: combined_financial_data_short.csv  
    * Columns: date, symbol, year, period, filing_date, financial variables, ...  
    * Size: 22488 × 126  
    * 862 unique companies  

FIX ME PROBABLY WRONG

In [64]:
# Whether this is a sample/debug run or not
DEBUG = False

In [65]:
# Packages
import pandas as pd
import openpyxl

In [66]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv
cr_and_dates = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv')
print(cr_and_dates)

     ticker          fixed_quarter_date earnings_call_date Rating  \
0      ABBV  2014-10-01 00:00:00.000000         2014-07-25      A   
1      ABBV  2015-01-01 00:00:00.000000         2014-10-31      A   
2      ABBV  2015-04-01 00:00:00.000000         2015-01-30      A   
3      ABBV  2015-07-01 00:00:00.000000         2015-04-23      A   
4      ABBV  2015-10-01 00:00:00.000000         2015-07-24      A   
...     ...                         ...                ...    ...   
4393    ZTS  2015-10-01 00:00:00.000000         2015-08-04    BBB   
4394    ZTS  2016-01-01 00:00:00.000000         2015-11-03    BBB   
4395    ZTS  2016-04-01 00:00:00.000000         2016-02-16    BBB   
4396    ZTS  2016-07-01 00:00:00.000000         2016-05-04    BBB   
4397    ZTS  2016-10-01 00:00:00.000000         2016-08-03    BBB   

                      Rating Agency Name rating_date         Source  \
0     Standard & Poor's Ratings Services  2014-07-18       Original   
1     Standard & Poor's Ratin

In [67]:
# check for duplicates on ticker by fixed_quarter_date
print(cr_and_dates[cr_and_dates.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call]
Index: []


In [68]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Calls\calls.csv
if DEBUG:
    calls = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\calls.csv', nrows = 10)
else:
    calls = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.csv')
# Delete the first column (index)
#calls = calls.iloc[:,1:]
# rename symbol to company
calls = calls.rename(columns = {'symbol':'company'})
# Convert date to be just the date part
calls['date'] = calls['date'].str[:10]
print(calls)

      company  quarter  year        date  \
0           F        1  2010  2010-04-27   
1           F        2  2010  2010-07-23   
2           F        3  2010  2010-10-26   
3           F        4  2010  2011-01-28   
4           F        1  2011  2011-04-26   
...       ...      ...   ...         ...   
31074    XPER        1  2015  2015-05-06   
31075    XPER        4  2016  2017-02-22   
31076    XPER        3  2016  2016-11-01   
31077    XPER        2  2016  2016-08-02   
31078    XPER        1  2016  2016-05-02   

                                                 content  source  
0      Operator: Good day, ladies and gentlemen, and ...  kaggle  
1      Operator: Good day, ladies and gentlemen, and ...  kaggle  
2      Operator: Good day, ladies and gentlemen, and ...  kaggle  
3      Operator: Good day ladies and gentlemen and we...  kaggle  
4      Operator: Good day, ladies and gentlemen, and ...  kaggle  
...                                                  ...     ...  
31

In [69]:
# check for duplicates on company by date
print(calls[calls.duplicated(subset=['company', 'date'], keep=False)])

      company  quarter  year        date  \
2560      URI        3  2013  2013-04-17   
4099      BBQ        3  2011  2011-08-19   
4100      BBQ        4  2011  2011-08-19   
19299    PCAR        4  2010  2011-04-21   
19306    PCAR        1  2011  2011-04-21   
21209     URI        1  2013  2013-04-17   
25182    ENTG        3  2015  2015-07-30   
25183    ENTG        2  2015  2015-07-30   
29070     WDC        4  2011  2011-07-21   
29071     WDC        2  2011  2011-07-21   
30536    QRVO        3  2011  2011-10-26   
30542    QRVO        2  2012  2011-10-26   
30996     WRK        4  2013  2014-01-29   
31003     WRK        1  2014  2014-01-29   

                                                 content  source  
2560   Operator: Good morning, and welcome to the Uni...  kaggle  
4099   Operator: [Operator Comments]\nDiana Purcel: G...  kaggle  
4100   Diana Purcel: Good morning everyone and thank ...  kaggle  
19299  Operator: Good morning, and welcome to PACCAR'...     web  
1930

### Earnings Call Data Merge

In [70]:
# Inner join cr_and_dates (on columns ticker, earnings_call_date) with calls.csv (on columns company, date)
merged = pd.merge(cr_and_dates, calls, left_on=['ticker', 'earnings_call_date'], right_on=['company', 'date'], how='inner')
merged

# Print range of dates
print('date range')
print(merged['date'].min())
print(merged['date'].max())

# Range of year, credit_rating_year
for variable in ['year', 'credit_rating_year']:
    print(variable)
    print(merged[variable].min())
    print(merged[variable].max())

# Print rows where year is greater than 2016
# This is due to inconsistencies in the earnings call data year variable
print('years greater than 2016')
print(merged[merged['year'] > 2016][['earnings_call_date', 'ticker', 'company', 'date', 'year', 'credit_rating_year']])

# Free up memory from calls
del calls

date range
2010-07-31
2016-09-30
year
2010
2016
credit_rating_year
2010
2016
years greater than 2016
Empty DataFrame
Columns: [earnings_call_date, ticker, company, date, year, credit_rating_year]
Index: []


In [71]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

     ticker          fixed_quarter_date earnings_call_date Rating  \
2553   PCAR  2011-07-01 00:00:00.000000         2011-04-21      A   
2554   PCAR  2011-07-01 00:00:00.000000         2011-04-21      A   
3422    URI  2013-07-01 00:00:00.000000         2013-04-17      B   
3423    URI  2013-07-01 00:00:00.000000         2013-04-17      B   

                      Rating Agency Name rating_date         Source  \
2553  Standard & Poor's Ratings Services  2011-05-31           Both   
2554  Standard & Poor's Ratings Services  2011-05-31           Both   
3422  Standard & Poor's Ratings Services  2012-02-22  Supplementary   
3423  Standard & Poor's Ratings Services  2012-02-22  Supplementary   

      Rating Rank AAA is 10 Next Rating Next Rating Date  ...  \
2553                      8           A       2011-12-21  ...   
2554                      8           A       2011-12-21  ...   
3422                      5           B       2013-11-26  ...   
3423                      5           

### Financial Statements Data Merge

In [72]:
# Load tabular financial statement data
df = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Tabular_Fin\combined_corrected_tabular_financial_statements_data.parquet')
print(df)

# Perform merge
merged = pd.merge(merged, df, left_on=['company', 'year', 'quarter'], right_on=['symbol', 'calendarYear', 'period'], how='inner')    
print(merged)

            date symbol reportedCurrency      cik fillingDate  \
0     2016-12-31    BEP              USD  1533232  2017-03-01   
1     2016-09-30    BEP              USD  1533232  2016-10-03   
2     2016-06-30    BEP              USD  1533232  2016-07-05   
3     2016-03-31    BEP              USD  1533232  2016-04-01   
4     2015-12-31    BEP              USD  1533232  2016-01-04   
...          ...    ...              ...      ...         ...   
20820 2011-03-31   YORW              USD   108985  2011-05-06   
20821 2010-12-31   YORW              USD   108985  2011-03-08   
20822 2010-09-30   YORW              USD   108985  2010-11-05   
20823 2010-06-30   YORW              USD   108985  2010-08-06   
20824 2010-03-31   YORW              USD   108985  2010-05-07   

              acceptedDate  calendarYear  period  cashAndCashEquivalents  \
0      2017-03-01 06:27:55          2016       4             223000000.0   
1      2016-10-03 06:45:16          2016       3             232000

In [73]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

     ticker          fixed_quarter_date earnings_call_date Rating  \
2531   PCAR  2011-07-01 00:00:00.000000         2011-04-21      A   
2532   PCAR  2011-07-01 00:00:00.000000         2011-04-21      A   
3388    URI  2013-07-01 00:00:00.000000         2013-04-17      B   
3389    URI  2013-07-01 00:00:00.000000         2013-04-17      B   

                      Rating Agency Name rating_date         Source  \
2531  Standard & Poor's Ratings Services  2011-05-31           Both   
2532  Standard & Poor's Ratings Services  2011-05-31           Both   
3388  Standard & Poor's Ratings Services  2012-02-22  Supplementary   
3389  Standard & Poor's Ratings Services  2012-02-22  Supplementary   

      Rating Rank AAA is 10 Next Rating Next Rating Date  ...     marketCap  \
2531                      8           A       2011-12-21  ...  2.097680e+10   
2532                      8           A       2011-12-21  ...  1.916035e+10   
3388                      5           B       2013-11-26  ...

### Rating Changes

In [74]:
# Variable for rating on previous fixed_quarter_date
# Sort by ticker, fixed_quarter_date
merged = merged.sort_values(by=['ticker', 'fixed_quarter_date'])
# Shift the rating by 1
merged['rating_on_previous_fixed_quarter_date'] = merged.groupby('ticker')['Rating'].shift(1)
merged[['ticker', 'fixed_quarter_date', 'Rating', 'rating_on_previous_fixed_quarter_date']].head(10)

,ticker,fixed_quarter_date,Rating,rating_on_previous_fixed_quarter_date
0,ABBV,2014-10-01 00:00:00.000000,A,NaN
1,ABBV,2015-01-01 00:00:00.000000,A,A
2,ABBV,2015-04-01 00:00:00.000000,A,A
3,ABBV,2015-07-01 00:00:00.000000,A,A
4,ABBV,2015-10-01 00:00:00.000000,A,A
5,ABBV,2016-01-01 00:00:00.000000,A,A
6,ABBV,2016-04-01 00:00:00.000000,A,A
7,ABBV,2016-07-01 00:00:00.000000,A,A
8,ABBV,2016-10-01 00:00:00.000000,A,A
9,ABC,2011-04-01 00:00:00.000000,A,NaN


In [75]:
# Rating change variables
# Get numeric values for Rating and rating_on_next_earnings_call_date
# Encode Ratings as ordinal
rating_order = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'CC', 'C', 'D']
# Reverse
rating_order = rating_order[::-1]
# Create dictionary mapping to number values
rating_dict = {rating: i for i, rating in enumerate(rating_order, 1)}
# Create new column "Rating Rank AAA is 10"
merged['Rating Rank AAA is 10'] = merged['Rating'].map(rating_dict)
# Flag for investment grade - Rating Rank >= 7.0
merged['Investment_Grade'] = merged['Rating Rank AAA is 10'] >= 7.0
# Create new column "rating_on_previous_fixed_quarter_date AAA is 10"
merged['rating_on_previous_fixed_quarter_date AAA is 10'] = merged['rating_on_previous_fixed_quarter_date'].map(rating_dict)

# Cross tab Rating and 'Rating Rank AAA is 10'
print(pd.crosstab(merged['Rating'], merged['Rating Rank AAA is 10']))
# Cross tab rating_on_previous_fixed_quarter_date and 'rating_on_previous_fixed_quarter_date AAA is 10'
print(pd.crosstab(merged['rating_on_previous_fixed_quarter_date'], merged['rating_on_previous_fixed_quarter_date AAA is 10']))

Rating Rank AAA is 10  2   3    4    5    6     7    8    9   10
Rating                                                          
A                       0   0    0    0    0     0  706    0   0
AA                      0   0    0    0    0     0    0  168   0
AAA                     0   0    0    0    0     0    0    0  78
B                       0   0    0  448    0     0    0    0   0
BB                      0   0    0    0  900     0    0    0   0
BBB                     0   0    0    0    0  1210    0    0   0
C                      13   0    0    0    0     0    0    0   0
CC                      0   2    0    0    0     0    0    0   0
CCC                     0   0  113    0    0     0    0    0   0
rating_on_previous_fixed_quarter_date AAA is 10  2.0   3.0   4.0   5.0   6.0   \
rating_on_previous_fixed_quarter_date                                           
A                                                   0     0     0     0     0   
AA                                        

In [76]:
# Variable for change before next call
def change(row):
    if row['Rating Rank AAA is 10'] < row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Downgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] > row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Upgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] == row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Same As Last Fixed Quarter Date'
    else:
        return None

# Apply function to create new column
merged['Change Direction Since Last Fixed Quarter Date'] = merged.apply(change, axis = 1)

# Also numeric version 'Change Since Last Fixed Quarter Date'
merged['Change Since Last Fixed Quarter Date'] = merged['Rating Rank AAA is 10'] - merged['rating_on_previous_fixed_quarter_date AAA is 10']

# Cross tab 'Change Direction Since Last Fixed Quarter Date' and 'Change Since Last Fixed Quarter Date'
print(pd.crosstab(merged['Change Direction Since Last Fixed Quarter Date'], merged['Change Since Last Fixed Quarter Date']))

Change Since Last Fixed Quarter Date            -2.0  -1.0   0.0   1.0   2.0
Change Direction Since Last Fixed Quarter Date                              
Downgrade Since Last Fixed Quarter Date           10    79     0     0     0
Same As Last Fixed Quarter Date                    0     0  3128     0     0
Upgrade Since Last Fixed Quarter Date              0     0     0   104    13


### Sector and Other Textual Company Information

From Kaggle: https://www.kaggle.com/datasets/aramacus/usa-public-companies

Supplemented with manually looked up sector data for missing sectors

In [77]:
# Load CSV 'C:\Users\ijyli\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv'
combined_sector_data = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv')
print(combined_sector_data)

     Ticker                                        Description  \
0      CTVA  Corteva, Inc. is a global provider of seed and...   
1      ALCO  Alico, Inc. is an agribusiness and land manage...   
2      LMNR  Limoneira Company is primarily an agribusiness...   
3      SANW  S&W Seed Company (S&W) is a multi-crop and mid...   
4       TRC  Tejon Ranch Co. is a diversified real estate d...   
...     ...                                                ...   
3384   RELX                                                NaN   
3385    TAL                                                NaN   
3386   TRTN                                                NaN   
3387   VLRS                                                NaN   
3388    XIN                                                NaN   

        Company Name                  Sector            Industry Group  \
0        CORTEVA INC               Materials                 Materials   
1          ALICO INC        Consumer Staples  Food, Beverag

In [78]:
# Left join us_stocks with merged
merged = pd.merge(merged, combined_sector_data, left_on='ticker', right_on='Ticker', how='left')
print(merged)

     ticker          fixed_quarter_date earnings_call_date Rating  \
0      ABBV  2014-10-01 00:00:00.000000         2014-07-25      A   
1      ABBV  2015-01-01 00:00:00.000000         2014-10-31      A   
2      ABBV  2015-04-01 00:00:00.000000         2015-01-30      A   
3      ABBV  2015-07-01 00:00:00.000000         2015-04-23      A   
4      ABBV  2015-10-01 00:00:00.000000         2015-07-24      A   
...     ...                         ...                ...    ...   
3633    ZTS  2015-10-01 00:00:00.000000         2015-08-04    BBB   
3634    ZTS  2016-01-01 00:00:00.000000         2015-11-03    BBB   
3635    ZTS  2016-04-01 00:00:00.000000         2016-02-16    BBB   
3636    ZTS  2016-07-01 00:00:00.000000         2016-05-04    BBB   
3637    ZTS  2016-10-01 00:00:00.000000         2016-08-03    BBB   

                      Rating Agency Name rating_date         Source  \
0     Standard & Poor's Ratings Services  2014-07-18       Original   
1     Standard & Poor's Ratin

In [79]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

     ticker          fixed_quarter_date earnings_call_date Rating  \
2531   PCAR  2011-07-01 00:00:00.000000         2011-04-21      A   
2532   PCAR  2011-07-01 00:00:00.000000         2011-04-21      A   
3388    URI  2013-07-01 00:00:00.000000         2013-04-17      B   
3389    URI  2013-07-01 00:00:00.000000         2013-04-17      B   

                      Rating Agency Name rating_date         Source  \
2531  Standard & Poor's Ratings Services  2011-05-31           Both   
2532  Standard & Poor's Ratings Services  2011-05-31           Both   
3388  Standard & Poor's Ratings Services  2012-02-22  Supplementary   
3389  Standard & Poor's Ratings Services  2012-02-22  Supplementary   

      Rating Rank AAA is 10 Next Rating Next Rating Date  ...  \
2531                      8           A       2011-12-21  ...   
2532                      8           A       2011-12-21  ...   
3388                      5           B       2013-11-26  ...   
3389                      5           

In [80]:
# Print if ticker = PCAR and fixed_quarter_date = 2011-07-01
# Compare fixed_quarter_date to datetime of 2011-07-01
#print(merged[(merged['ticker'] == 'PCAR') & (merged['fixed_quarter_date'] == pd.to_datetime('2011-07-01'))])

### Some Basic Stats

In [81]:
# Print range of dates
# print('date range')
# print(merged['date'].min())
# print(merged['date'].max())

# Range of year, credit_rating_year, calendarYear
# for variable in ['year', 'credit_rating_year', 'calendarYear']:
#     print(variable)
#     print(merged[variable].min())
#     print(merged[variable].max())

In [82]:
# Summarize columns
print(merged.info())
# print column names
print(merged.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3638 entries, 0 to 3637
Columns: 169 entries, ticker to Comment
dtypes: bool(1), datetime64[ns](1), float64(114), int32(1), int64(9), object(43)
memory usage: 4.7+ MB
None
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'Source', 'Rating Rank AAA is 10',
       'Next Rating', 'Next Rating Date',
       ...
       'Change Direction Since Last Fixed Quarter Date',
       'Change Since Last Fixed Quarter Date', 'Ticker', 'Description',
       'Company Name', 'Sector', 'Industry Group', 'Industry', 'Sub-Industry',
       'Comment'],
      dtype='object', length=169)


In [83]:
# Save a parquet file as well
merged.to_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates.parquet', index=False)

In [ ]:
# Sample 100 rows to create all_data_sample.csv
if not DEBUG:
    all_data_sample = merged.sample(100)
    all_data_sample.to_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.csv', index=False)
    # also save to xlsx
    all_data_sample.to_excel(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.xlsx', index=False)